# **1. Generative Models for Text**

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from keras.utils import np_utils
from sklearn.cluster import KMeans
from skimage.color import rgb2gray
#import cv2
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Conv2D, Flatten, BatchNormalization, Activation, Dropout,LeakyReLU, add, MaxPooling2D,LSTM
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras import regularizers
from keras.optimizers import Adam,SGD
opt = Adam(lr=0.1)
from keras.models import model_from_json

**(b) Download the following books from Project Gutenberg http://www.gutenberg. org/ebooks/author/355 in text format:
i. The Problems of Philosophy
ii. The Analysis of Mind
iii. Mysticism and Logic and Other Essays
iv. Our Knowledge of the External World as a Field for Scientific Method in Philosophy
Project Gutenberg adds a standard header and footer to each book and this is not part of the original text. Open the file in a text editor and delete the header and footer.
The header is obvious and ends with the text:
*** START OF THIS PROJECT GUTENBERG EBOOK AN INQUIRY INTO MEANING AND TRUTH ***
The footer is all of the text after the line of text that says:
THE END
To have a better model, it is strongly recommended that you download the fol- lowing books from The Library of Congress https://archive.org and convert them to text files:
i. The History of Western Philosophy
             https://archive.org/details/westernphilosophy4
ii. The Analysis of Matter
             https://archive.org/details/in.ernet.dli.2015.221533
iii. An Inquiry into Meaning and Truth
            https://archive.org/details/BertrandRussell-AnInquaryIntoMeaningAndTruth
Try to only use the text of the books and throw away unwanted text before and after the text, although in a large corpus, these are considered as noise and should not make big problems.1 **

In [0]:
filenames=os.listdir('drive/assgn-6')
#filenames=filenames.remove('.Trash')
filenames=['drive/assgn-6/'+i for i in filenames]
import string

In [0]:
with open('drive/final.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [0]:
import re
line = re.sub('[!@#$]', '', line)
f=open('drive/final.txt','r')
x=f.read().lower()
chars = sorted(list(set(x))) 
yo=['\n',' ',',','.','0', '1', '2', '3', '4', '5', '6', '7', '8' ,'9','a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
chars1=chars
for i in yo:
  chars1.remove(i)
for i in chars1:
  x=x.replace(i,"")
x1=str()
for i in chars1:
  x1=x1+i  
x = re.sub('[)+/;>[_| ·æéîôüŭβηιλορτὴίό″]', '', x)
x=x.replace("\n"," ")
#chars.remove("\n")
x=' '.join(x.split())


### **Answer**:All the text files were inserted into a directory from which they were concatenated to a single file called 'final'. Only letters of alphabet and spaces with comma and period were kept as characters. New line was also removed.

## c) LSTM: Train an LSTM to mimic Russell’s style and thoughts:
## i. Concatenate your text files to create a corpus of Russell’s writings.
## ii. Use a character-level representation for this model by using extended ASCII that has N = 256 characters. Each character will be encoded into a an integer using its ASCII code. Rescale the integers to the range [0, 1], because LSTM uses a sigmoid activation function. LSTM will receive the rescaled integers as its input

In [0]:
chars = sorted(list(set(x))) 
with open('drive/final1.txt', 'w') as outfile:
    outfile.write(x)

## iii) Choose a window size, e.g., W = 100.

## iv)Inputs to the network will be the first W −1 = 99 characters of each sequence, and the output of the network will be the Lth character of the sequence. Basically, we are training the network to predict the each character using the 99 characters that precede it. Slide the window in strides of S = 1 on the text. For example, if W = 5 and S = 1 and we want to train the network with the sequence ABRACADABRA, The first input to the network will be ABRA and the corresponding output will be C. The second input will be BRAC and the second output will be A, etc.

In [0]:
with open('drive/final1.txt', 'r') as file:
    x=file.read()
x=x[:int(len(x))]    
unique_chars = sorted(list(set(x))) 
char_ascii = dict((c, i) for i, c in enumerate(unique_chars))  

In [0]:
n_chars = len(x)
n_vocab = len(unique_chars)
window = 99
input_seq = []
output_seq = []
for i in range(0, n_chars - window,1):
    input1 = x[i:i + window]
    output1 =x[i+window]
    input_seq.append([char_ascii[char] for char in input1])
    output_seq.append(char_ascii[output1])
n_patterns = len(input_seq)
X = np.reshape(input_seq, (n_patterns, window, 1))

In [0]:
X = X / float(n_vocab)

Answer: Sequences of length 99 were created and output is of length 1. All these sequences then act as input to the LSTM. 
Also, the input characters were rescaled between [0,1] to make use of sigmoid function.

## v)Note that the output has to be encoded using a one-hot encoding scheme with N = 256 (or less) elements. This means that the network reads integers, but outputs a vector of N = 256 (or less) elements.

Also, the final output is one-hot encoded to make use of softmax.

In [0]:
output_seq=np.array(output_seq)
y = np_utils.to_categorical(output_seq)


## vi)Use a single hidden layer for the LSTM with N = 256 (or less) memory units.
## vii)Use a Softmax output layer to yield a probability prediction for each of the characters between 0 and 1. This is actually a character classification problem with N classes. Choose log loss (cross entropy) as the objective function for the network (research what it menas).3
## viii)We do not use a test dataset. We are using the whole training dataset to learn the probability of each character in a sequence. We are not seeking for a very accurate model of. Instead we are interested in a generalization of the dataset that can mimic the gist of the text.

In [0]:
reverse_map = dict((i, c) for i, c in enumerate(unique_chars))
model = Sequential()
model.add(LSTM(100, input_shape=(X.shape[1], X.shape[2]), activation='sigmoid',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.load_weights("drive/weights-improvement-10-2.1577-bigger.hdf5")
model.compile(loss='categorical_crossentropy', optimizer='adam')

#### Answer: I used a single hidden layer. The categorical cross-entropy was used.
#### The cross entropy between two probability distributions  p and  q over the same underlying set of events measures the average number of bits needed to identify an event drawn from the set, if a coding scheme is used that is optimized for an "unnatural" probability distribution  q, rather than the "true" distribution  p.

## ix)Choose a reasonable number of epochs for training (e.g., 30, although the network will need more epochs to yield a better model).
## x)Use model checkpointing to keep the network weights to determine each time an improvement in loss is observed at the end of the epoch. Find the best set of weights in terms of loss.

#### Answer : At the end of each epoch we store the best set of weights with ModelCheckpoint.

In [0]:
filepath="drive/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [11]:
model.fit(X,y,epochs=10,batch_size=256,callbacks=callbacks_list)
model.save_weights("drive/model.h5")

Epoch 1/10
1320348/1320348 [==============================] - 2427s 2ms/step - loss: 2.3090

Epoch 00001: loss improved from inf to 2.30896, saving model to drive/weights-improvement-01-2.3090-bigger.hdf5
Epoch 2/10
 127488/1320348 [=>............................] - ETA: 36:36 - loss: 2.28471320348/1320348 [==============================] - 2431s 2ms/step - loss: 2.2825

Epoch 00002: loss improved from 2.30896 to 2.28254, saving model to drive/weights-improvement-02-2.2825-bigger.hdf5
Epoch 3/10
 124928/1320348 [=>............................] - ETA: 36:47 - loss: 2.26801320348/1320348 [==============================] - 2429s 2ms/step - loss: 2.2609

Epoch 00003: loss improved from 2.28254 to 2.26091, saving model to drive/weights-improvement-03-2.2609-bigger.hdf5
Epoch 4/10
 124928/1320348 [=>............................] - ETA: 36:26 - loss: 2.25001320348/1320348 [==============================] - 2420s 2ms/step - loss: 2.2406

Epoch 00004: loss improved from 2.26091 to 2.24055, savi

#### Answer: The model was run for approximately 26 epochs . The runtime was significant- 39 minutes for 1 epoch. This is the reason I could only run for 18 epochs. Only ten epochs are shown here but the weights for each epoch were saved to a file, only if there was improvement.

# xi) Use the network with the best weights to generate 1000 characters, using the following text as initialization of the network:
There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.

In [21]:
init_text = "There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.".lower()
init_text=[char_ascii[char] for char in init_text]
for i in range(1000):
	x1 = np.reshape(init_text, (1, len(init_text), 1))
	x1 = x1 / float(n_vocab)
	prediction = model.predict(x1, verbose=0)
	index = np.argmax(prediction)
	result = reverse_map[index]
	seq_in = [reverse_map[value] for value in init_text]
	print(result)
	init_text.append(index)
	init_text = init_text[1:len(init_text)]

l
e
a
n
d
t
h
e
s
e
i
s
a
n
o
r
e
a
r
e
n
o
t
i
o
n
s
o
f
t
h
e
s
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
t
h
e
m
a
t
i
o
n
s
,
t
h
e
r
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
a
r
e
n
o
t
i
o
n
s
o
f
t
h
e
s
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
t
h
e
m
a
t
i
o
n
s
,
t
h
e
r
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
a
r
e
n
o
t
i
o
n
s
o
f
t
h
e
s
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
t
h
e
m
a
t
i
o
n
s
,
t
h
e
r
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
a
r
e
n
o
t
i
o
n
s
o
f
t
h
e
s
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
t
h
e
m
a
t
i
o
n
s
,
t
h
e
r
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
a
r
e
n
o
t
i
o
n
s
o
f
t
h
e
s
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
t
h
e
m
a
t
i
o
n
s
,
t
h
e
r
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
a
r
e
n
o
t
i
o
n
s
o
f
t
h
e
s
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
t
h
e
m
a
t
i
o
n
s
,
t
h
e
r
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
a
r
e
n
o
t
i
o
n
s
o
f
t
h
e
s
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
t
h
e
m
a
t
i
o
n
s
,
t
h
e
r
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
a
r
e
n
o
t
i
o
n
s
o
f
t
h
e
s
e
a
r
e
t
h
e
s
e
i
s
a
n
o
r
e
t
h
e
m
a
t
i
o
n
s


#### Answer: As we can see Thousand characters were generated. Although the model still requires training as it  was run for 18 epochs only. The model started understanding the basic grammar and was able to use comma and period successfully.